In [20]:
import sys
sys.path.append('..')

from helper_fun import *
import helper_fun

# Silence some expected warnings
filterwarnings("ignore")

from Split_functions_regression.split_furthest_cluster import *
from Split_functions_regression.split_hierarchical_cluster import *
from Split_functions_regression.split_random import *
from Split_functions_regression.split_strat_pIC50 import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import seaborn as sns

# Neural network specific libraries
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint


SEED = 42
seed_everything(SEED)

# Set path to this notebook
HERE = Path(_dh[-1])
DATA = HERE / "data"

compounds = pd.read_csv('../COVID_MOONSHOT/compounds_filtered.csv')

,Unnamed: 0,SMILES,CID,f_avg_IC50,r_avg_IC50,ROMol,molecular_weight,n_hba,n_hbd,logp,Ro5_fulfilled,f_avg_pIC50,r_avg_pIC50
0,35,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1,DAR-DIA-23aa0b97-19,26.719515,14.641091,<rdkit.Chem.rdchem.Mol object at 0x7fc319ac34c0>,287.105862,3,1,3.28768,True,4.573171,4.834427
1,36,O=C(Cc1cncc2ccccc12)Nc1ccccc1,DAR-DIA-23aa0b97-20,57.590417,45.077469,<rdkit.Chem.rdchem.Mol object at 0x7fc319ac3530>,262.110613,2,1,3.41600,True,4.239650,4.346040


In [21]:
compounds["Fingerprints"] = compounds["SMILES"].apply(smiles_to_fp)
x_train_rand, x_test_rand, y_train_rand, y_test_rand = random_split(compounds["Fingerprints"], compounds["f_avg_pIC50"])
x_train_strat, x_test_strat, y_train_strat, y_test_strat = strat_pIC50_split(compounds)
x_train_hi, x_test_hi, y_train_hi, y_test_hi = split_hierarchical_clusters(compounds, test_size=0.2, random_state=42)
x_train_noise, x_test_noise, y_train_noise, y_test_noise = UMAP_noise_split(compounds)
x_train_fur, x_test_fur, y_train_fur, y_test_fur = furthest_cluster_split(compounds)
# print(f"Shape of train_x: {train_x.shape}") (476,)
# print(f"Shape of test_x: {test_y.shape}") (119,)


Assign cluster ID


  0%|          | 0/595 [00:00<?, ?it/s][11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPRECATION WARNING: please use MorganGenerator
[11:45:33] DEPREC

In [28]:
rfr_rand = RandomForestRegressor(random_state=SEED)
rfr_strat = RandomForestRegressor(random_state=SEED)
rfr_hi = RandomForestRegressor(random_state=SEED)
rfr_noise = RandomForestRegressor(random_state=SEED)
rfr_fur = RandomForestRegressor(random_state=SEED)
rfr_rand.fit(np.array(list((x_train_rand))).astype(float), y_train_rand)
rfr_strat.fit(np.array(list((x_train_strat))).astype(float), y_train_strat)
rfr_hi.fit(np.array(list((x_train_hi))).astype(float), y_train_hi)
rfr_noise.fit(np.array(list((x_train_noise))).astype(float), y_train_noise)
rfr_fur.fit(np.array(list((x_train_fur))).astype(float), y_train_fur)

RandomForestRegressor(random_state=42)

In [29]:
y_pred_rfr_rand = rfr_rand.predict(np.array(list((x_test_rand))).astype(float))
y_pred_rfr_strat = rfr_strat.predict(np.array(list((x_test_strat))).astype(float))
y_pred_rfr_hi = rfr_hi.predict(np.array(list((x_test_hi))).astype(float))
y_pred_rfr_noise = rfr_noise.predict(np.array(list((x_test_noise))).astype(float))
y_pred_rfr_fur = rfr_fur.predict(np.array(list((x_test_fur))).astype(float))

In [31]:
mae_rfr_rand = mean_absolute_error(y_test_rand, y_pred_rfr_rand)
mse_rfr_rand = mean_squared_error(y_test_rand, y_pred_rfr_rand)
r2_rfr_rand = r2_score(y_test_rand, y_pred_rfr_rand)

mae_rfr_strat = mean_absolute_error(y_test_strat, y_pred_rfr_strat)
mse_rfr_strat = mean_squared_error(y_test_strat, y_pred_rfr_strat)
r2_rfr_strat = r2_score(y_test_strat, y_pred_rfr_strat)

mae_rfr_hi = mean_absolute_error(y_test_hi, y_pred_rfr_hi)
mse_rfr_hi = mean_squared_error(y_test_hi, y_pred_rfr_hi)
r2_rfr_hi = r2_score(y_test_hi, y_pred_rfr_hi)

mae_rfr_noise = mean_absolute_error(y_test_noise, y_pred_rfr_noise)
mse_rfr_noise = mean_squared_error(y_test_noise, y_pred_rfr_noise)
r2_rfr_noise = r2_score(y_test_noise, y_pred_rfr_noise)

mae_rfr_fur = mean_absolute_error(y_test_fur, y_pred_rfr_fur)
mse_rfr_fur = mean_squared_error(y_test_fur, y_pred_rfr_fur)
r2_rfr_fur = r2_score(y_test_fur, y_pred_rfr_fur)

In [32]:
#Creat a table that shows the performance of the random forest regressor on the different splits displaying mae, mse and r2 values
split_scores_rfr_initial = {'Split': ['Random', 'Stratified pIC50', 'Hierarchical Clustering', 'UMAP Noise', 'UMAP Furthest Cluster'],
        'MAE': [mae_rfr_rand, mae_rfr_strat, mae_rfr_hi, mae_rfr_noise, mae_rfr_fur],
        'MSE': [mse_rfr_rand, mse_rfr_strat, mse_rfr_hi, mse_rfr_noise, mse_rfr_fur],
        'R2': [r2_rfr_rand, r2_rfr_strat, r2_rfr_hi, r2_rfr_noise, r2_rfr_fur]}

split_scores_rfr_initial_df = pd.DataFrame(split_scores_rfr_initial)
split_scores_rfr_initial_df.set_index("Split", inplace=True)
#save df to csv in data file
split_scores_rfr_initial_df.to_csv(DATA / "split_scores_rfr_initial.csv")

print(split_scores_rfr_initial_df)

                              MAE       MSE        R2
Split                                                
Random                   0.382970  0.309122  0.579999
Stratified pIC50         0.461976  0.423172  0.516725
Hierarchical Clustering  0.674829  0.705286  0.323519
UMAP Noise               0.787113  0.913533 -0.179115
UMAP Furthest Cluster    0.839491  0.938845 -0.237629


# Optuna

In [48]:
def objective(trial, X_train, y_train):
    n_estimators = trial.suggest_int("n_estimators", 2, 500)
    max_depth = trial.suggest_int("max_depth", 2, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 50)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 50)

    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1).mean()
    return score

In [68]:
study_rand = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED))
study_strat = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED))
study_hi = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED))
study_noise = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED))
study_fur = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED))


[I 2024-07-29 13:00:28,100] A new study created in memory with name: no-name-d9d00627-1541-46d0-96ca-5f8bc2a00891
[I 2024-07-29 13:00:28,103] A new study created in memory with name: no-name-bf7e876b-7118-4952-b140-f797665b6a62
[I 2024-07-29 13:00:28,106] A new study created in memory with name: no-name-6cba7155-7f3e-48b4-a320-bddf5edc9f4e
[I 2024-07-29 13:00:28,109] A new study created in memory with name: no-name-6143c822-6e46-4f3c-9f50-66999845e895
[I 2024-07-29 13:00:28,111] A new study created in memory with name: no-name-c0c8d958-71d5-41c7-b910-4b432f912eb2


In [69]:
#add trials using study.optimize
study_rand.optimize(lambda trial: objective(trial, np.array(list((x_train_rand))).astype(float), y_train_rand), n_trials=200, show_progress_bar=True)

  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-29 13:02:00,247] Trial 0 finished with value: -0.6684983150560477 and parameters: {'n_estimators': 188, 'max_depth': 48, 'min_samples_split': 37, 'min_samples_leaf': 30}. Best is trial 0 with value: -0.6684983150560477.
[I 2024-07-29 13:02:00,761] Trial 1 finished with value: -0.7239100458278541 and parameters: {'n_estimators': 79, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 44}. Best is trial 0 with value: -0.6684983150560477.
[I 2024-07-29 13:02:01,971] Trial 2 finished with value: -0.7366020211370621 and parameters: {'n_estimators': 301, 'max_depth': 36, 'min_samples_split': 3, 'min_samples_leaf': 49}. Best is trial 0 with value: -0.6684983150560477.
[I 2024-07-29 13:02:04,286] Trial 3 finished with value: -0.529789797686855 and parameters: {'n_estimators': 417, 'max_depth': 12, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 3 with value: -0.529789797686855.
[I 2024-07-29 13:02:05,056] Trial 4 finished with value: -0.5772483072029229 and p

In [70]:

study_strat.optimize(lambda trial: objective(trial, np.array(list((x_train_strat))).astype(float), y_train_strat), n_trials=200, show_progress_bar=True)


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-29 13:05:45,407] Trial 0 finished with value: -0.6441660272739511 and parameters: {'n_estimators': 188, 'max_depth': 48, 'min_samples_split': 37, 'min_samples_leaf': 30}. Best is trial 0 with value: -0.6441660272739511.
[I 2024-07-29 13:05:45,734] Trial 1 finished with value: -0.6999763900413865 and parameters: {'n_estimators': 79, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 44}. Best is trial 0 with value: -0.6441660272739511.
[I 2024-07-29 13:05:46,981] Trial 2 finished with value: -0.7119319769361262 and parameters: {'n_estimators': 301, 'max_depth': 36, 'min_samples_split': 3, 'min_samples_leaf': 49}. Best is trial 0 with value: -0.6441660272739511.
[I 2024-07-29 13:05:49,193] Trial 3 finished with value: -0.5010318009388504 and parameters: {'n_estimators': 417, 'max_depth': 12, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 3 with value: -0.5010318009388504.
[I 2024-07-29 13:05:49,889] Trial 4 finished with value: -0.5491613096621354 and

In [71]:
study_hi.optimize(lambda trial: objective(trial, np.array(list((x_train_hi))).astype(float), y_train_hi), n_trials=200, show_progress_bar=True)


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-29 13:09:25,545] Trial 0 finished with value: -0.7167219274670849 and parameters: {'n_estimators': 188, 'max_depth': 48, 'min_samples_split': 37, 'min_samples_leaf': 30}. Best is trial 0 with value: -0.7167219274670849.
[I 2024-07-29 13:09:25,876] Trial 1 finished with value: -0.7477847211586006 and parameters: {'n_estimators': 79, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 44}. Best is trial 0 with value: -0.7167219274670849.
[I 2024-07-29 13:09:27,017] Trial 2 finished with value: -0.7617869269563752 and parameters: {'n_estimators': 301, 'max_depth': 36, 'min_samples_split': 3, 'min_samples_leaf': 49}. Best is trial 0 with value: -0.7167219274670849.
[I 2024-07-29 13:09:28,868] Trial 3 finished with value: -0.6053018149299224 and parameters: {'n_estimators': 417, 'max_depth': 12, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 3 with value: -0.6053018149299224.
[I 2024-07-29 13:09:29,592] Trial 4 finished with value: -0.6424199499177167 and

In [72]:
study_noise.optimize(lambda trial: objective(trial, np.array(list((x_train_noise))).astype(float), y_train_noise), n_trials=200, show_progress_bar=True)


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-29 13:12:26,446] Trial 0 finished with value: -1.5071943232138791 and parameters: {'n_estimators': 188, 'max_depth': 48, 'min_samples_split': 37, 'min_samples_leaf': 30}. Best is trial 0 with value: -1.5071943232138791.
[I 2024-07-29 13:12:26,604] Trial 1 finished with value: -1.5053346379045176 and parameters: {'n_estimators': 79, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 44}. Best is trial 1 with value: -1.5053346379045176.
[I 2024-07-29 13:12:27,167] Trial 2 finished with value: -1.5043201719368686 and parameters: {'n_estimators': 301, 'max_depth': 36, 'min_samples_split': 3, 'min_samples_leaf': 49}. Best is trial 2 with value: -1.5043201719368686.
[I 2024-07-29 13:12:28,187] Trial 3 finished with value: -0.9171519059553427 and parameters: {'n_estimators': 417, 'max_depth': 12, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 3 with value: -0.9171519059553427.
[I 2024-07-29 13:12:28,490] Trial 4 finished with value: -1.490152867383379 and 

In [73]:
study_fur.optimize(lambda trial: objective(trial, np.array(list((x_train_fur))).astype(float), y_train_fur), n_trials=200, show_progress_bar=True)

  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-07-29 13:14:16,351] Trial 0 finished with value: -1.5169363729938703 and parameters: {'n_estimators': 188, 'max_depth': 48, 'min_samples_split': 37, 'min_samples_leaf': 30}. Best is trial 0 with value: -1.5169363729938703.
[I 2024-07-29 13:14:16,510] Trial 1 finished with value: -1.501407584090083 and parameters: {'n_estimators': 79, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 44}. Best is trial 1 with value: -1.501407584090083.
[I 2024-07-29 13:14:17,082] Trial 2 finished with value: -1.4968018732497892 and parameters: {'n_estimators': 301, 'max_depth': 36, 'min_samples_split': 3, 'min_samples_leaf': 49}. Best is trial 2 with value: -1.4968018732497892.
[I 2024-07-29 13:14:17,926] Trial 3 finished with value: -1.2643938586039054 and parameters: {'n_estimators': 417, 'max_depth': 12, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 3 with value: -1.2643938586039054.
[I 2024-07-29 13:14:18,238] Trial 4 finished with value: -1.4584903448030198 and p

In [74]:
best_params_rand = study_rand.best_params
best_params_strat = study_strat.best_params
best_params_hi = study_hi.best_params
best_params_noise = study_noise.best_params
best_params_fur = study_fur.best_params


In [75]:
best_n_estimators_rand = best_params_rand["n_estimators"]
best_max_depth_rand = best_params_rand["max_depth"]
best_min_samples_split_rand = best_params_rand["min_samples_split"]
best_min_samples_leaf_rand = best_params_rand["min_samples_leaf"]

best_n_estimators_strat = best_params_strat["n_estimators"]
best_max_depth_strat = best_params_strat["max_depth"]
best_min_samples_split_strat = best_params_strat["min_samples_split"]
best_min_samples_leaf_strat = best_params_strat["min_samples_leaf"]

best_n_estimators_hi = best_params_hi["n_estimators"]
best_max_depth_hi = best_params_hi["max_depth"]
best_min_samples_split_hi = best_params_hi["min_samples_split"]
best_min_samples_leaf_hi = best_params_hi["min_samples_leaf"]

best_n_estimators_noise = best_params_noise["n_estimators"]
best_max_depth_noise = best_params_noise["max_depth"]
best_min_samples_split_noise = best_params_noise["min_samples_split"]
best_min_samples_leaf_noise = best_params_noise["min_samples_leaf"]

best_n_estimators_fur = best_params_fur["n_estimators"]
best_max_depth_fur = best_params_fur["max_depth"]
best_min_samples_split_fur = best_params_fur["min_samples_split"]
best_min_samples_leaf_fur = best_params_fur["min_samples_leaf"]

In [76]:

rfr_rand_best = RandomForestRegressor(random_state=SEED, n_estimators=best_n_estimators_rand, max_depth=best_max_depth_rand, min_samples_split=best_min_samples_split_rand, min_samples_leaf=best_min_samples_leaf_rand)
rfr_strat_best = RandomForestRegressor(random_state=SEED, n_estimators=best_n_estimators_strat, max_depth=best_max_depth_strat, min_samples_split=best_min_samples_split_strat, min_samples_leaf=best_min_samples_leaf_strat)
rfr_hi_best = RandomForestRegressor(random_state=SEED, n_estimators=best_n_estimators_hi, max_depth=best_max_depth_hi, min_samples_split=best_min_samples_split_hi, min_samples_leaf=best_min_samples_leaf_hi)
rfr_noise_best = RandomForestRegressor(random_state=SEED, n_estimators=best_n_estimators_noise, max_depth=best_max_depth_noise, min_samples_split=best_min_samples_split_noise, min_samples_leaf=best_min_samples_leaf_noise)
rfr_fur_best = RandomForestRegressor(random_state=SEED, n_estimators=best_n_estimators_fur, max_depth=best_max_depth_fur, min_samples_split=best_min_samples_split_fur, min_samples_leaf=best_min_samples_leaf_fur)

rfr_rand_best.fit(np.array(list((x_train_rand))).astype(float), y_train_rand)
rfr_strat_best.fit(np.array(list((x_train_strat))).astype(float), y_train_strat)
rfr_hi_best.fit(np.array(list((x_train_hi))).astype(float), y_train_hi)
rfr_noise_best.fit(np.array(list((x_train_noise))).astype(float), y_train_noise)
rfr_fur_best.fit(np.array(list((x_train_fur))).astype(float), y_train_fur)

y_pred_rfr_rand_best = rfr_rand_best.predict(np.array(list((x_test_rand))).astype(float))
y_pred_rfr_strat_best = rfr_strat_best.predict(np.array(list((x_test_strat))).astype(float))
y_pred_rfr_hi_best = rfr_hi_best.predict(np.array(list((x_test_hi))).astype(float))
y_pred_rfr_noise_best = rfr_noise_best.predict(np.array(list((x_test_noise))).astype(float))
y_pred_rfr_fur_best = rfr_fur_best.predict(np.array(list((x_test_fur))).astype(float))

In [77]:
                                          
mae_rfr_rand_best = mean_absolute_error(y_test_rand, y_pred_rfr_rand_best)
mae_rfr_strat_best = mean_absolute_error(y_test_strat, y_pred_rfr_strat_best)
mae_rfr_hi_best = mean_absolute_error(y_test_hi, y_pred_rfr_hi_best)
mae_rfr_noise_best = mean_absolute_error(y_test_noise, y_pred_rfr_noise_best)
mae_rfr_fur_best = mean_absolute_error(y_test_fur, y_pred_rfr_fur_best)

mse_rfr_rand_best = mean_squared_error(y_test_rand, y_pred_rfr_rand_best)
mse_rfr_strat_best = mean_squared_error(y_test_strat, y_pred_rfr_strat_best)
mse_rfr_hi_best = mean_squared_error(y_test_hi, y_pred_rfr_hi_best)
mse_rfr_noise_best = mean_squared_error(y_test_noise, y_pred_rfr_noise_best)
mse_rfr_fur_best = mean_squared_error(y_test_fur, y_pred_rfr_fur_best)

r2_rfr_rand_best = r2_score(y_test_rand, y_pred_rfr_rand_best)
r2_rfr_strat_best = r2_score(y_test_strat, y_pred_rfr_strat_best)
r2_rfr_hi_best = r2_score(y_test_hi, y_pred_rfr_hi_best)
r2_rfr_noise_best = r2_score(y_test_noise, y_pred_rfr_noise_best)
r2_rfr_fur_best = r2_score(y_test_fur, y_pred_rfr_fur_best)

split_scores_rfr_best = {'Split': ['Random', 'Random Optimized', 'Stratified pIC50', 'Stratified pIC50 Optimized', 'Hierarchical Clustering', 'Hierarchical Clustering Optimized', 'UMAP Noise', 'UMAP Noise Optimized', 'UMAP Furthest Cluster', 'UMAP Furthest Cluster Optimized'],
        'MAE': [mae_rfr_rand, mae_rfr_rand_best, mae_rfr_strat, mae_rfr_strat_best, mae_rfr_hi, mae_rfr_hi_best, mae_rfr_noise, mae_rfr_noise_best, mae_rfr_fur, mae_rfr_fur_best],
        'MSE': [mse_rfr_rand, mse_rfr_rand_best, mse_rfr_strat, mse_rfr_strat_best, mse_rfr_hi, mse_rfr_hi_best, mse_rfr_noise, mse_rfr_noise_best, mse_rfr_fur, mse_rfr_fur_best],
        'R2': [r2_rfr_rand, r2_rfr_rand_best, r2_rfr_strat, r2_rfr_strat_best, r2_rfr_hi, r2_rfr_hi_best, r2_rfr_noise, r2_rfr_noise_best, r2_rfr_fur, r2_rfr_fur_best]}
split_scores_rfr_best_df = pd.DataFrame(split_scores_rfr_best)
print(split_scores_rfr_best_df)

                               Split       MAE       MSE        R2
0                             Random  0.382970  0.309122  0.579999
1                   Random Optimized  0.393117  0.310572  0.578029
2                   Stratified pIC50  0.461976  0.423172  0.516725
3         Stratified pIC50 Optimized  0.504089  0.459178  0.475605
4            Hierarchical Clustering  0.674829  0.705286  0.323519
5  Hierarchical Clustering Optimized  0.681877  0.710977  0.318060
6                         UMAP Noise  0.787113  0.913533 -0.179115
7               UMAP Noise Optimized  0.784079  0.897608 -0.158560
8              UMAP Furthest Cluster  0.839491  0.938845 -0.237629
9    UMAP Furthest Cluster Optimized  0.876427  1.058782 -0.395735


In [63]:
optuna.visualization.plot_optimization_history(study)

### Dark lines are parameters that work better

In [64]:
optuna.visualization.plot_parallel_coordinate(study)

In [67]:
optuna.visualization.plot_slice(study, params=["n_estimators", "max_depth", "min_samples_split", "min_samples_leaf"])

In [66]:
optuna.visualization.plot_param_importances(study)